In [1]:
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms

from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

import numpy as np
from time import time
import copy

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

CUDA = torch.cuda.is_available()

In [3]:
RAW_DATA_DIR = "../Data/Raw/"
PROCESSED_DATA_DIR = "../Data/Processed/Regression/"

class CovariateDataset(Dataset):
    def __init__(self, file_name_pattern, file_name_args, train_size=0.8, test_size=0.2, test_train_complement=True):
        self.train = True
        self.test_on_all = False
        
        self.file_name = file_name_pattern.format(*file_name_args, "covar")
        self.assignment_file_name = file_name_pattern.format(*file_name_args, "assignment")
        
        self.data = np.loadtxt(RAW_DATA_DIR + self.file_name + ".csv", delimiter=",")[:, 1:] # remove bias
        self.assignment_data = np.loadtxt(
            RAW_DATA_DIR + self.assignment_file_name + ".csv", delimiter=",").astype(int)
        
        self.all_indeces = np.array(range(len(self.data)))
        treat_indeces = self.all_indeces[self.assignment_data.astype(int) == 1]
        control_indeces = self.all_indeces[self.assignment_data.astype(int) == 0]
        
        num_training = int(len(self.data)*train_size)
        
        self.train_indeces = np.random.choice(self.all_indeces, num_training, replace=False)
        if test_train_complement:
            self.test_indeces = list(set(self.all_indeces)^set(self.train_indeces))      
        else:
            self.test_indeces = np.random.choice(self.all_indeces, int(len(self.data)*(1-test_size)), replace=False)
        
        num_treated_in_train = len(np.intersect1d(treat_indeces, self.train_indeces, assume_unique=True))
        num_control_in_train = num_training - num_treated_in_train
        
        treat_weight = num_training / (2 * num_treated_in_train)
        control_weight = num_training / (2 * num_control_in_train)
        
        weighter = np.vectorize(lambda index: treat_weight if index in\
            treat_indeces else control_weight)
        
        self.weights = weighter(self.all_indeces)
        
    def active_data(self, index=0):
        if self.train:
            return self.data[self.train_indeces], self.assignment_data[self.train_indeces], \
                self.weights[self.train_indeces][index]
        else:
            if self.test_on_all:
                indeces = self.all_indeces
            else: 
                indeces = self.test_indeces
            
            return self.data[indeces], self.assignment_data[indeces], 1
            
    def __getitem__(self, index):
        covar_data, assignment_data, weight_data = self.active_data(index)
        class_vector = np.zeros(2)
        class_vector[int(assignment_data[index])] = 1
        
        return (covar_data[index], class_vector, weight_data)

    def __len__(self):
        return self.active_data()[0].shape[0]
    
    def save_processed_data(self, data):
        name = PROCESSED_DATA_DIR + self.file_name+".csv"
        np.savetxt(name, data, delimiter=",")
        
def get_datasets(file_name_format, file_name_args, **kwargs):
    train_set = CovariateDataset(file_name_format, file_name_args, **kwargs)
    test_set = copy.deepcopy(train_set)
    test_set.train = False

    predict_set = copy.deepcopy(train_set)
    predict_set.train = False
    predict_set.test_on_all = True
    
    return train_set, test_set, predict_set

In [4]:
# Based on an example from https://github.com/pytorch/examples/blob/master/vae/main.py
# Extended to place a different prior on binary vs normal vars

class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()
        
        INTERMEDIATE_DIMS_1 = 16
        INTERMEDIATE_DIMS_2 = 16
        INTERMEDIATE_DIMS_3 = 16
        INTERMEDIATE_DIMS_4 = 16
#         INTERMEDIATE_DIMS_5 = 16
#         INTERMEDIATE_DIMS_6 = 8

        FEATURES = 10

        LOSS_SCALE = 1

        # ENCODER LAYERS
        self.dense1 = nn.Linear(FEATURES, INTERMEDIATE_DIMS_1)
        self.dense2 = nn.Linear(INTERMEDIATE_DIMS_1, INTERMEDIATE_DIMS_2)
        self.dense3 = nn.Linear(INTERMEDIATE_DIMS_2, INTERMEDIATE_DIMS_3)
        self.dense4 = nn.Linear(INTERMEDIATE_DIMS_3, INTERMEDIATE_DIMS_4)
#         self.dense5 = nn.Linear(INTERMEDIATE_DIMS_4, INTERMEDIATE_DIMS_5)
#         self.dense6 = nn.Linear(INTERMEDIATE_DIMS_5, INTERMEDIATE_DIMS_6)
        self.dense5 = nn.Linear(INTERMEDIATE_DIMS_4, 2)
        
        # Activations
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.6)

    def forward(self, x):
        h1 = self.dropout(self.relu(self.dense1(x)))
        h2 = self.dropout(self.relu(self.dense2(h1)))
        h3 = self.dropout(self.relu(self.dense3(h2)))
        h4 = self.dropout(self.relu(self.dense4(h3)))
#         h5 = self.dropout(self.relu(self.dense5(h4)))
#         h6 = self.dropout(self.relu(self.dense6(h5)))
        
        return self.softmax(self.dense5(h4))



def train(model, optimizer, epoch, train_loader, log_results=False):
    model.train()
    train_loss = 0

    for batch_idx, (data, target_class, weights) in enumerate(train_loader):
        data = Variable(data)
        target_class = Variable(target_class)
        weights = Variable(weights)
        
        data = data.float()
        target_class = target_class.float()
        weights = weights.float()
        
        if CUDA:
            data = data.cuda()
            target_class = target_class.cuda()
            weights = weights.cuda()
        
        optimizer.zero_grad()

        output_propensity = model(data)
        
        # calculate loss
        loss_criterion = nn.BCELoss(weight=weights.view(weights.shape[0], 1), size_average=False)
        loss = loss_criterion(output_propensity, target_class)

        train_loss += loss.data[0]
        
        # Find the gradient and descend
        loss.backward()
        optimizer.step()
        
    if log_results:
        print('====> Epoch: {} Average loss: {:.8f}'.format(
              epoch, train_loss / len(train_loader.dataset)))
        
        
def test(model, epoch, test_loader):
    # toggle model to test / inference mode
    model.eval()
    test_loss = 0

    for i, (data, target_class, weights) in enumerate(test_loader):
        data = Variable(data, volatile=True)
        target_class = Variable(target_class, volatile=True)
        weights = Variable(weights, volatile=True)
        
        data = data.float()
        target_class = target_class.float()
        weights = weights.float()
        
        if CUDA:
            data = data.cuda()
            target_class = target_class.cuda()
            weights = weights.cuda()

        output_propensity = model(data)
        
        # calculate loss
        loss_criterion = nn.BCELoss(weight=weights.view(weights.shape[0], 1), size_average=False)
        loss = loss_criterion(output_propensity, target_class)
        test_loss += loss.data[0]

    test_loss /= len(test_loader.dataset)
    
    if CUDA:
        output_propensity = output_propensity.cpu()
        target_class = target_class.cpu()
        
    score = accuracy(output_propensity.data.numpy(), target_class.data.numpy(), verbose=False)
    print('====> Test set loss: {:.4f}, {}%'.format(test_loss, score*100))
    
def predict(model, predict_loader):
    # Show reconstruction
    model.eval()
    print("Training state: ", model.training)
    
    original_data, targets, _ = next(iter(predict_loader))
    
    original_data = Variable(original_data)
    original_data = original_data.float()
    
    if CUDA:
        original_data = original_data.cuda()
        
    return original_data, targets, model(original_data)

def accuracy(output_data, targets, verbose=True):
        
    classes = np.argmax(output_data, axis=1)
    targets = np.argmax(targets, axis=1)
    
    if verbose:
        print(classification_report(targets, classes))
    return accuracy_score(targets, classes)

In [5]:
def train_model(model_class, train_set, test_set, predict_set, dataset_number, verbose=True, model=None):
    if model is None:
        model = model_class()
        if CUDA:
            model = model.cuda()

    num_epochs = 750
    train_batch_size = 64
    test_batch_size = 250
    learning_rate = 1e-3
    lr_sched = True
         
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [int(num_epochs/5), int(num_epochs/2)], gamma=0.1)

    train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=test_batch_size, shuffle=True)
    predict_loader = DataLoader(predict_set, batch_size=1000, shuffle=False)
    
    for epoch in range(1, num_epochs+1):
        checkpoint_interval = int(num_epochs/10)
        
        if lr_sched:
            scheduler.step()

        log = False
        if epoch%checkpoint_interval == 0:
            log = True
            
        train(model, optimizer, epoch, train_loader, log_results=log)
        if log:
            test(model, epoch, test_loader)
    
    original_data, targets, output = predict(model, predict_loader)
    if CUDA:
        output = output.cpu()
        targets = targets.cpu()
    
    return model, original_data, targets, output

def encode_data(dataset, output_data):
    
    if CUDA:
        output_data = output_data.cpu()
        
    dataset.save_processed_data(output_data.data.numpy()[:, 1])


In [7]:
def run_logistic(train_set, verbose=True):
    model = LogisticRegression(class_weight="balanced")
    
    X = train_set.data
    y = train_set.assignment_data

    X_train = X[train_set.train_indeces]
    X_test = X[train_set.test_indeces]
    y_train = y[train_set.train_indeces]
    y_test = y[train_set.test_indeces]

    model.fit(X_train, y_train)
    predictions = model.predict(X)
    
    if verbose:
        print(classification_report(y, predictions))
    
    return accuracy_score(y, predictions)

In [8]:
train_set, test_set, predict_set = get_datasets(
    "n_{}_model_{}_v_{}_{}_data", [1000, "G_mod_nadd_mod_nlin", 1],
    train_size=0.8, test_train_complement=True)

start = time()
trained_model, original_data, targets, output = \
    train_model(Regressor, train_set, test_set,predict_set, 1,verbose=True)
print("Elapsed: ", time() - start)


acc = accuracy(output.data.cpu().numpy(), targets.numpy(), verbose=False)
print("Complete set accuracy: {}%".format(acc*100))

encode_data(train_set, output)

====> Epoch: 75 Average loss: 1.32145905
====> Test set loss: 1.2942, 62.0%
====> Epoch: 150 Average loss: 1.25285402
====> Test set loss: 1.2176, 68.5%
====> Epoch: 225 Average loss: 1.28917585
====> Test set loss: 1.2106, 68.5%
====> Epoch: 300 Average loss: 1.23299732
====> Test set loss: 1.2073, 68.0%
====> Epoch: 375 Average loss: 1.24430372
====> Test set loss: 1.1989, 68.0%
====> Epoch: 450 Average loss: 1.23220974
====> Test set loss: 1.1982, 68.0%
====> Epoch: 525 Average loss: 1.26749196
====> Test set loss: 1.1987, 68.0%
====> Epoch: 600 Average loss: 1.24656528
====> Test set loss: 1.1988, 68.0%
====> Epoch: 675 Average loss: 1.25521823
====> Test set loss: 1.2001, 68.0%
====> Epoch: 750 Average loss: 1.26520532
====> Test set loss: 1.1999, 68.0%
Training state:  False
Elapsed:  26.38823103904724
Complete set accuracy: 68.7%


### Runner

In [11]:
assignment_model_names = ['A_add_lin', 'B_add_mild_nlin', 'C_add_mod_nlin', 'D_mild_nadd_lin',
                     'E_mild_nadd_mild_nlin', 'F_mod_nadd_lin', 'G_mod_nadd_mod_nlin']

nn_accuracies = []
log_accuracies = []

for dataset_number in range(0, 50):
    print("Starting run for Dataset {}".format(dataset_number))
    
    for model_name in assignment_model_names:
        print("---- Running for model name: ", model_name)
        
        start = time()

        train_set, test_set, predict_set = get_datasets(
            "n_{}_model_{}_v_{}_{}_data", [1000, model_name, dataset_number], train_size=0.8)

        trained_model, original_data, targets, output = \
            train_model(Regressor, train_set, test_set,predict_set, 1,verbose=True)
        
        nn_acc = accuracy(output.data.cpu().numpy(), targets.numpy(), verbose=False)
        print("Complete set accuracy: {}%".format(nn_acc*100))
        
        log_acc = run_logistic(train_set, verbose=False)
        print("Log accuracy: {}%".format(log_acc*100))
        
        nn_accuracies.append(nn_acc)
        log_accuracies.append(log_acc)

        encode_data(train_set, output)

        print("---- Done in ", time() - start, " seconds\n")
                
    print("================\n\n")

Starting run for Dataset 0
---- Running for model name:  A_add_lin
====> Epoch: 75 Average loss: 1.25209245
====> Test set loss: 1.2033, 72.0%
====> Epoch: 150 Average loss: 1.23347114
====> Test set loss: 1.1826, 68.0%
====> Epoch: 225 Average loss: 1.20111577
====> Test set loss: 1.1705, 68.0%
====> Epoch: 300 Average loss: 1.21055777
====> Test set loss: 1.1666, 68.0%
====> Epoch: 375 Average loss: 1.17975743
====> Test set loss: 1.1677, 68.0%
====> Epoch: 450 Average loss: 1.20965408
====> Test set loss: 1.1670, 68.0%
====> Epoch: 525 Average loss: 1.19567036
====> Test set loss: 1.1654, 68.0%
====> Epoch: 600 Average loss: 1.20057317
====> Test set loss: 1.1653, 68.0%
====> Epoch: 675 Average loss: 1.19405594
====> Test set loss: 1.1656, 68.0%
====> Epoch: 750 Average loss: 1.22272699
====> Test set loss: 1.1665, 68.0%
Training state:  False
Complete set accuracy: 72.39999999999999%
Log accuracy: 70.6%
---- Done in  50.24499702453613  seconds

---- Running for model name:  B_add_m

====> Epoch: 675 Average loss: 1.17681717
====> Test set loss: 1.0728, 76.0%
====> Epoch: 750 Average loss: 1.17936993
====> Test set loss: 1.0725, 76.0%
Training state:  False
Complete set accuracy: 73.5%
Log accuracy: 70.3%
---- Done in  58.94323205947876  seconds

---- Running for model name:  C_add_mod_nlin
====> Epoch: 75 Average loss: 1.34492078
====> Test set loss: 1.2330, 65.5%
====> Epoch: 150 Average loss: 1.21927511
====> Test set loss: 1.1799, 67.5%
====> Epoch: 225 Average loss: 1.19561067
====> Test set loss: 1.1804, 67.5%
====> Epoch: 300 Average loss: 1.19071211
====> Test set loss: 1.1757, 67.5%
====> Epoch: 375 Average loss: 1.25650632
====> Test set loss: 1.1756, 67.0%
====> Epoch: 450 Average loss: 1.22016419
====> Test set loss: 1.1754, 67.0%
====> Epoch: 525 Average loss: 1.21895984
====> Test set loss: 1.1749, 67.0%
====> Epoch: 600 Average loss: 1.20075268
====> Test set loss: 1.1743, 67.0%
====> Epoch: 675 Average loss: 1.23627179
====> Test set loss: 1.1738, 6

====> Epoch: 450 Average loss: 1.16545919
====> Test set loss: 1.0652, 75.0%
====> Epoch: 525 Average loss: 1.16844955
====> Test set loss: 1.0655, 75.0%
====> Epoch: 600 Average loss: 1.13536200
====> Test set loss: 1.0650, 75.0%
====> Epoch: 675 Average loss: 1.13338516
====> Test set loss: 1.0640, 75.0%
====> Epoch: 750 Average loss: 1.14549058
====> Test set loss: 1.0633, 75.5%
Training state:  False
Complete set accuracy: 77.5%
Log accuracy: 74.7%
---- Done in  51.69770288467407  seconds

---- Running for model name:  E_mild_nadd_mild_nlin
====> Epoch: 75 Average loss: 1.20177260
====> Test set loss: 1.1404, 73.0%
====> Epoch: 150 Average loss: 1.18663787
====> Test set loss: 1.0828, 73.5%
====> Epoch: 225 Average loss: 1.14059043
====> Test set loss: 1.0804, 74.0%
====> Epoch: 300 Average loss: 1.13106277
====> Test set loss: 1.0807, 73.5%
====> Epoch: 375 Average loss: 1.10917548
====> Test set loss: 1.0780, 73.5%
====> Epoch: 450 Average loss: 1.12146091
====> Test set loss: 1.

====> Epoch: 225 Average loss: 1.21300390
====> Test set loss: 1.1046, 73.0%
====> Epoch: 300 Average loss: 1.18423234
====> Test set loss: 1.1016, 74.5%
====> Epoch: 375 Average loss: 1.22068683
====> Test set loss: 1.0998, 75.0%
====> Epoch: 450 Average loss: 1.18175865
====> Test set loss: 1.0997, 75.0%
====> Epoch: 525 Average loss: 1.23069819
====> Test set loss: 1.0991, 75.0%
====> Epoch: 600 Average loss: 1.17263702
====> Test set loss: 1.0995, 75.0%
====> Epoch: 675 Average loss: 1.22747241
====> Test set loss: 1.0996, 75.0%
====> Epoch: 750 Average loss: 1.21947708
====> Test set loss: 1.0995, 75.0%
Training state:  False
Complete set accuracy: 75.1%
Log accuracy: 72.6%
---- Done in  51.35845398902893  seconds

---- Running for model name:  G_mod_nadd_mod_nlin
====> Epoch: 75 Average loss: 1.21276071
====> Test set loss: 1.1570, 71.5%
====> Epoch: 150 Average loss: 1.13068509
====> Test set loss: 1.1262, 72.5%
====> Epoch: 225 Average loss: 1.17879767
====> Test set loss: 1.14

====> Epoch: 75 Average loss: 1.26681994
====> Test set loss: 1.1853, 73.5%
====> Epoch: 150 Average loss: 1.18781907
====> Test set loss: 1.1158, 73.0%
====> Epoch: 225 Average loss: 1.19898814
====> Test set loss: 1.1147, 73.0%
====> Epoch: 300 Average loss: 1.17566998
====> Test set loss: 1.1123, 72.5%
====> Epoch: 375 Average loss: 1.24409288
====> Test set loss: 1.1084, 73.0%
====> Epoch: 450 Average loss: 1.17719968
====> Test set loss: 1.1079, 72.5%
====> Epoch: 525 Average loss: 1.20768282
====> Test set loss: 1.1075, 72.5%
====> Epoch: 600 Average loss: 1.19273392
====> Test set loss: 1.1071, 72.5%
====> Epoch: 675 Average loss: 1.17173861
====> Test set loss: 1.1073, 72.5%
====> Epoch: 750 Average loss: 1.22235806
====> Test set loss: 1.1072, 72.5%
Training state:  False
Complete set accuracy: 72.39999999999999%
Log accuracy: 72.7%
---- Done in  50.55862808227539  seconds

---- Running for model name:  B_add_mild_nlin
====> Epoch: 75 Average loss: 1.27888889
====> Test set lo

...

====> Epoch: 75 Average loss: 1.26897005
====> Test set loss: 1.2614, 65.0%
====> Epoch: 150 Average loss: 1.23538420
====> Test set loss: 1.1564, 72.0%
====> Epoch: 225 Average loss: 1.22269173
====> Test set loss: 1.1534, 71.5%
====> Epoch: 300 Average loss: 1.23111382
====> Test set loss: 1.1536, 71.5%
====> Epoch: 375 Average loss: 1.18852023
====> Test set loss: 1.1498, 71.0%
====> Epoch: 450 Average loss: 1.16265339
====> Test set loss: 1.1493, 71.0%
====> Epoch: 525 Average loss: 1.18549775
====> Test set loss: 1.1502, 71.0%
====> Epoch: 600 Average loss: 1.21107455
====> Test set loss: 1.1498, 71.0%
====> Epoch: 675 Average loss: 1.24080701
====> Test set loss: 1.1496, 71.0%
====> Epoch: 750 Average loss: 1.19682404
====> Test set loss: 1.1497, 71.0%
Training state:  False
Complete set accuracy: 73.2%
Log accuracy: 68.4%
---- Done in  61.46671199798584  seconds



Starting run for Dataset 44
---- Running for model name:  A_add_lin
====> Epoch: 75 Average loss: 1.18954703
====> 

====> Epoch: 750 Average loss: 1.18803387
====> Test set loss: 1.2702, 61.5%
Training state:  False
Complete set accuracy: 68.4%
Log accuracy: 71.5%
---- Done in  67.02608394622803  seconds

---- Running for model name:  B_add_mild_nlin
====> Epoch: 75 Average loss: 1.27914188
====> Test set loss: 1.2548, 63.0%
====> Epoch: 150 Average loss: 1.21039824
====> Test set loss: 1.2173, 66.5%
====> Epoch: 225 Average loss: 1.24948722
====> Test set loss: 1.2235, 66.0%
====> Epoch: 300 Average loss: 1.22683923
====> Test set loss: 1.2247, 67.0%
====> Epoch: 375 Average loss: 1.18251556
====> Test set loss: 1.2227, 66.5%
====> Epoch: 450 Average loss: 1.18938235
====> Test set loss: 1.2230, 66.5%
====> Epoch: 525 Average loss: 1.22699142
====> Test set loss: 1.2234, 66.5%
====> Epoch: 600 Average loss: 1.21406773
====> Test set loss: 1.2234, 66.5%
====> Epoch: 675 Average loss: 1.22608233
====> Test set loss: 1.2239, 66.5%
====> Epoch: 750 Average loss: 1.23670406
====> Test set loss: 1.2226, 

====> Epoch: 525 Average loss: 1.16946878
====> Test set loss: 1.0805, 72.5%
====> Epoch: 600 Average loss: 1.15258520
====> Test set loss: 1.0808, 72.0%
====> Epoch: 675 Average loss: 1.13274683
====> Test set loss: 1.0808, 72.0%
====> Epoch: 750 Average loss: 1.14021974
====> Test set loss: 1.0806, 72.0%
Training state:  False
Complete set accuracy: 76.6%
Log accuracy: 72.39999999999999%
---- Done in  63.44416904449463  seconds

---- Running for model name:  D_mild_nadd_lin
====> Epoch: 75 Average loss: 1.25657752
====> Test set loss: 1.1267, 78.0%
====> Epoch: 150 Average loss: 1.19757657
====> Test set loss: 1.0877, 75.5%
====> Epoch: 225 Average loss: 1.20733292
====> Test set loss: 1.0791, 75.0%
====> Epoch: 300 Average loss: 1.22971477
====> Test set loss: 1.0774, 75.5%
====> Epoch: 375 Average loss: 1.23292642
====> Test set loss: 1.0746, 74.5%
====> Epoch: 450 Average loss: 1.20955904
====> Test set loss: 1.0742, 74.5%
====> Epoch: 525 Average loss: 1.17636807
====> Test set l

====> Epoch: 300 Average loss: 1.18881411
====> Test set loss: 1.1305, 70.5%
====> Epoch: 375 Average loss: 1.19411249
====> Test set loss: 1.1308, 69.5%
====> Epoch: 450 Average loss: 1.20329071
====> Test set loss: 1.1307, 69.5%
====> Epoch: 525 Average loss: 1.19542566
====> Test set loss: 1.1304, 69.5%
====> Epoch: 600 Average loss: 1.18810572
====> Test set loss: 1.1306, 69.5%
====> Epoch: 675 Average loss: 1.17610447
====> Test set loss: 1.1306, 69.5%
====> Epoch: 750 Average loss: 1.19450677
====> Test set loss: 1.1302, 69.5%
Training state:  False
Complete set accuracy: 75.3%
Log accuracy: 71.8%
---- Done in  64.41713309288025  seconds

---- Running for model name:  F_mod_nadd_lin
====> Epoch: 75 Average loss: 1.23558473
====> Test set loss: 1.1779, 76.0%
====> Epoch: 150 Average loss: 1.19318219
====> Test set loss: 1.1376, 71.5%
====> Epoch: 225 Average loss: 1.17824158
====> Test set loss: 1.1317, 71.5%
====> Epoch: 300 Average loss: 1.20147741
====> Test set loss: 1.1325, 7

====> Epoch: 75 Average loss: 1.26799796
====> Test set loss: 1.2981, 66.0%
====> Epoch: 150 Average loss: 1.22731709
====> Test set loss: 1.2968, 63.0%
====> Epoch: 225 Average loss: 1.21538490
====> Test set loss: 1.3011, 62.0%
====> Epoch: 300 Average loss: 1.25556473
====> Test set loss: 1.3034, 62.5%
====> Epoch: 375 Average loss: 1.20234744
====> Test set loss: 1.3049, 62.5%
====> Epoch: 450 Average loss: 1.20696343
====> Test set loss: 1.3056, 62.5%
====> Epoch: 525 Average loss: 1.18810316
====> Test set loss: 1.3059, 62.5%
====> Epoch: 600 Average loss: 1.22545004
====> Test set loss: 1.3057, 62.5%
====> Epoch: 675 Average loss: 1.19947944
====> Test set loss: 1.3060, 62.5%
====> Epoch: 750 Average loss: 1.17288386
====> Test set loss: 1.3062, 62.5%
Training state:  False
Complete set accuracy: 73.4%
Log accuracy: 67.80000000000001%
---- Done in  67.95388698577881  seconds



Starting run for Dataset 49
---- Running for model name:  A_add_lin
====> Epoch: 75 Average loss: 1.27